## Fake News Classifier Using LSTM

About Dataset - Build a system to identify unreliable and Fake news articles

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks the article as potentially unreliable

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/train/train.csv') 

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [ ]:
df.isnull().sum() # Dataset have null or nan value

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.shape

(18285, 5)

In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
## Get the Dependent features
y=df['label']

In [ ]:
X.shape, y.shape

((18285, 4), (18285,))

### Data Preprocessing

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

The Porter stemming algorithm (or 'Porter stemmer') is a process for removing the commoner morphological and inflexional endings from words in English.

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
### Dataset Preprocessing

corpus = []
for i in range(0, len(messages)):
#     print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])    #clean the data
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[:10]   

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.7.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

### One Hot Representation

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:10]

[[1704, 1020, 188, 3507, 151, 135, 4830, 505, 1181, 4910],
 [2414, 2970, 1785, 1551, 2037, 3891, 825],
 [4814, 3030, 4729, 1391],
 [1806, 16, 1370, 2147, 399, 1276],
 [1520, 2037, 155, 3318, 3599, 1828, 2037, 1959, 4895, 2425],
 [3449,
  3056,
  2675,
  4026,
  519,
  1888,
  1020,
  1698,
  798,
  611,
  1251,
  540,
  2874,
  1545,
  825],
 [2031, 2246, 1608, 455, 443, 1006, 1052, 501, 3561, 1417, 214],
 [1443, 4380, 1993, 1853, 3138, 3522, 1888, 3042, 3561, 1417, 214],
 [496, 1290, 1494, 472, 4848, 2188, 2251, 4769, 1888, 564],
 [4627, 2786, 4489, 2073, 989, 739, 2914, 2106]]

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  505 1181 4910]
 [   0    0    0 ... 2037 3891  825]
 [   0    0    0 ... 3030 4729 1391]
 ...
 [   0    0    0 ... 3561 1417  214]
 [   0    0    0 ...  277 2250 1952]
 [   0    0    0 ... 3042 3912 1861]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1704,
       1020,  188, 3507,  151,  135, 4830,  505, 1181, 4910], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
#converted feature into numpy array
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12250, 20), (6035, 20), (12250,), (6035,))

### Model Training

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=64)

Epoch 1/20
192/192 [==============================] - 8s 16ms/step - loss: 0.3304 - accuracy: 0.8404 - val_loss: 0.1914 - val_accuracy: 0.9208
Epoch 2/20
192/192 [==============================] - 2s 12ms/step - loss: 0.1387 - accuracy: 0.9478 - val_loss: 0.1946 - val_accuracy: 0.9208
Epoch 3/20
192/192 [==============================] - 3s 13ms/step - loss: 0.0916 - accuracy: 0.9642 - val_loss: 0.2136 - val_accuracy: 0.9218
Epoch 4/20
192/192 [==============================] - 2s 12ms/step - loss: 0.0629 - accuracy: 0.9781 - val_loss: 0.2632 - val_accuracy: 0.9185
Epoch 5/20
192/192 [==============================] - 2s 12ms/step - loss: 0.0390 - accuracy: 0.9863 - val_loss: 0.3074 - val_accuracy: 0.9110
Epoch 6/20
192/192 [==============================] - 3s 14ms/step - loss: 0.0194 - accuracy: 0.9935 - val_loss: 0.3863 - val_accuracy: 0.9180
Epoch 7/20
192/192 [==============================] - 3s 13ms/step - loss: 0.0123 - accuracy: 0.9961 - val_loss: 0.4807 - val_accuracy: 0.9165

In [ ]:
# import os
# model_version=max([int(i) for i in os.listdir("/content/models") + [0]])+1
# model.save(f"/content/models/{model_version}")

In [ ]:
# model.save("/content/models/model1.h5")

In [ ]:
# !zip -r models.zip models/

In [ ]:
# filename = 'FakeNews-Classifier-RNN-model.pkl'
# pickle.dump(model, open(filename, 'wb'))

### Performance Metrics And Accuracy

In [ ]:
X_test

array([[   0,    0,    0, ..., 1573, 3064, 4351],
       [   0,    0,    0, ..., 3964, 2630,  672],
       [   0,    0,    0, ..., 3964, 2630,  672],
       ...,
       [   0,    0,    0, ..., 3964, 2630,  672],
       [   0,    0,    0, ...,    0,    0, 4294],
       [   0,    0,    0, ..., 4417, 1469, 2427]], dtype=int32)

In [ ]:
y_test

array([1, 0, 0, ..., 0, 1, 1])

In [ ]:
# y_pred=model.predict(X_test)

In [ ]:
y_pred1 = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
y_pred1

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

array([[3100,  319],
       [ 234, 2382]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)   #test accuracy

0.9083678541839271

In [ ]:
def predict(data):
  onehot_repr=[one_hot(words,voc_size)for words in data] 
  print(onehot_repr)
  sent_length=20
  embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

  return (model.predict(embedded_docs) > 0.5).astype("int32")


In [ ]:
def embedding(data):
  onehot_repr=[one_hot(words,voc_size)for words in data] 
  print(onehot_repr)
  sent_length=20
  
  return pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
import pickle

In [ ]:
# Creating a pickle file for the CountVectorizer
pickle.dump(embedding, open('embedding-transform.pkl', 'wb'))

In [ ]:
embedding = pickle.load(open('embedding-transform.pkl','rb'))

In [ ]:
data = embedding(['obama organ action partner soro link indivis disrupt trump agenda'])

[[2916, 1528, 1861, 4159, 1009, 1882, 4961, 3491, 4294, 1359]]


In [ ]:
from keras.models import load_model
model2 = load_model('my_model.h5')

In [ ]:
(model2.predict(data)>0.5).astype("int32")

array([[0]], dtype=int32)

In [ ]:
model1 = pickle.load(open('/content/FakeNews-Classifier-RNN-model.pkl','rb'))

In [ ]:
(model1.predict(data)>0.5).astype("int32")

array([[0]], dtype=int32)